<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# googledrive 연동
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# 모듈 임포트
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from scipy.io import wavfile
import librosa
import librosa.display

In [ ]:
# shifting
def shifting(data,direc,factor):
  result  = np.zeros_like(data)
  k = int(factor * (len(data)/10))
  #right
  if direc == '0':
    result[k:] = data[:len(data)-k]
  else:
    result[:len(data)-k] = data[k:]
  return result

#pith를 조절
def change_pitch(data,sr,k):
  return librosa.effects.pitch_shift(data,sr, k)

In [ ]:
def augmentation(data):
  from tqdm.notebook import tqdm
  new_data = []
  for i,cont in enumerate(tqdm(data)):
    hop_len =  int(22050/79)
    x, sr = librosa.load(cont)

    #shifting
    if i%3 == 0:
      direc = np.random.randint(0,1)
      shifted = librosa.feature.mfcc(shifting(x,direc,np.random.randint(1,2)),sr = sr, n_mfcc=80, fmax=3000, hop_length = hop_len)
      new_data.append(shifted.flatten())

    #pitch_change
    elif i%3 == 1:
      up_down = np.random.randint(0,1)
      if up_down == 0:
        pitch =  librosa.feature.mfcc(change_pitch(x,sr,2),sr = sr, n_mfcc=80, fmax=3000, hop_length =hop_len)
      else:
        pitch =  librosa.feature.mfcc(change_pitch(x,sr,-2),sr = %store, n_mfcc=80, fmax=3000, hop_length =hop_len)

      new_data.append(pitch.flatten())

    #both
    else:
      direc = np.random.randint(0,1)
      up_down = np.random.randint(0,1)

      if up_down == 0:
        move =  librosa.feature.mfcc(change_pitch(shifting(x,direc,np.random.randint(1,2)),sr,2),sr = sr, n_mfcc=80, fmax=3000, hop_length =hop_len)
      else:
        move =  librosa.feature.mfcc(change_pitch(shifting(x,direc,np.random.randint(1,2)),sr,-2),sr = %store, n_mfcc=80, fmax=3000, hop_length =hop_len)
      
      new_data.append(move.flatten())



  return pd.DataFrame(new_data)

if __name__ == '__main__':
  raw_data = glob('./train/*.wav')
  new_data  = augmentation(raw_data)
  new_data.to_pickle('drive/My Drive/data/x_trian_augmented_mfcc_80.pickle')